In [75]:
from til_environment import gridworld

env = gridworld.env()

In [76]:
from til_environment import gridworld

env = gridworld.env(
    env_wrappers=[],  # clear out default env wrappers
    render_mode="human",  # Render the map; not visible on Workbench
    debug=True,  # Enable debug mode
    novice=True,  # Use same map layout every time (for Novice teams only)
)

env.reset(seed=42)
score = 0

for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()

    if termination or truncation:
        break
    else:
        # Insert your policy here
        action = env.action_space(agent).sample()
        state = env.observation_space(agent)
    
    score += reward
    env.step(action)
    print(f"Action: {action}")
    print(f"State: {state}")

env.close()

Action: 3
State: Dict('direction': Discrete(4), 'location': Box(0, 16, (2,), uint8), 'scout': Discrete(2), 'step': Discrete(101), 'viewcone': Box(0, 255, (7, 5), uint8))
Action: 0
State: Dict('direction': Discrete(4), 'location': Box(0, 16, (2,), uint8), 'scout': Discrete(2), 'step': Discrete(101), 'viewcone': Box(0, 255, (7, 5), uint8))
Action: 2
State: Dict('direction': Discrete(4), 'location': Box(0, 16, (2,), uint8), 'scout': Discrete(2), 'step': Discrete(101), 'viewcone': Box(0, 255, (7, 5), uint8))
Action: 4
State: Dict('direction': Discrete(4), 'location': Box(0, 16, (2,), uint8), 'scout': Discrete(2), 'step': Discrete(101), 'viewcone': Box(0, 255, (7, 5), uint8))
Action: 3
State: Dict('direction': Discrete(4), 'location': Box(0, 16, (2,), uint8), 'scout': Discrete(2), 'step': Discrete(101), 'viewcone': Box(0, 255, (7, 5), uint8))
Action: 3
State: Dict('direction': Discrete(4), 'location': Box(0, 16, (2,), uint8), 'scout': Discrete(2), 'step': Discrete(101), 'viewcone': Box(0, 2

In [77]:
# multidiscrete
print(env)


til_env


# Stable Baselines 3 PPO

In [78]:
import functools
import numpy as np
import gymnasium as gym  # use `import gym` if using gym < 0.26
from pettingzoo.utils.env import AECEnv, AgentID, ObsType, ActionType
from pettingzoo.utils.wrappers.base import BaseWrapper


class CustomWrapper(BaseWrapper[AgentID, ObsType, ActionType], gym.Env):
    metadata = {"render_modes": ["human"]}

    def __init__(self, env: AECEnv[AgentID, ObsType, ActionType]):
        super().__init__(env)

        self.agents = self.env.possible_agents
        self.current_agent_index = 0
        self.current_agent = self.agents[self.current_agent_index]

        # Define Gym-style observation/action space
        self.observation_space = self.env.observation_space(self.current_agent)
        self.action_space = self.env.action_space(self.current_agent)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed, options=options)

        self.current_agent_index = 0
        self.current_agent = self.agents[self.current_agent_index]

        while self.env.agent_selection != self.current_agent:
            self.env.step(None)

        obs = self.observe(self.current_agent)
        return obs, {}

    def step(self, action: ActionType):
        reward = 0.0
        terminated = False
        truncated = False
        info = {}

        self.env.step(action)

        reward += self.env.rewards[self.current_agent]
        terminated = self.env.terminations[self.current_agent]
        truncated = self.env.truncations[self.current_agent]
        info = self.env.infos[self.current_agent]

        while (
            not self.env.terminations[self.current_agent]
            and not self.env.truncations[self.current_agent]
            and self.env.agent_selection != self.current_agent
        ):
            self.env.step(None)

        done = terminated or truncated
        obs = self.observe(self.current_agent)

        return obs, reward, done, truncated, info

    def render(self):
        return self.env.render()

    def close(self):
        return self.env.close()

    def observe(self, agent: AgentID) -> ObsType | None:
        return super().observe(agent)

    @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        return super().observation_space(agent)

    @functools.lru_cache(maxsize=None)
    def action_space(self, agent):
        return super().action_space(agent)


In [79]:
from til_environment.flatten_dict import FlattenDictWrapper
from til_environment import gridworld

env = gridworld.env(env_wrappers=[CustomWrapper])

In [80]:
from stable_baselines3 import PPO
model = PPO("CnnPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

Using cuda device


ValueError: The environment is of type <class 'pettingzoo.utils.wrappers.order_enforcing.OrderEnforcingWrapper'>, not a Gymnasium environment. In this case, we expect OpenAI Gym to be installed and the environment to be an OpenAI Gym environment.

# TorchRL PPO

In [149]:
# Torch
import torch
import torch.nn as nn

# Tensordict modules
from tensordict.nn import set_composite_lp_aggregate, TensorDictModule, TensorDictSequential
from tensordict import  TensorDictBase
from torch import multiprocessing

# Data collection
from torchrl.collectors import SyncDataCollector
from torch.distributions import Categorical
from torchrl.data.replay_buffers import ReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage

#Env
from torchrl.envs import RewardSum, TransformedEnv, PettingZooWrapper, Compose, DoubleToFloat, StepCounter, ParallelEnv, EnvCreator, ExplorationType, set_exploration_type

# Utils
from torchrl.envs.utils import check_env_specs

# Multi-agent network
from torchrl.modules import MultiAgentMLP, ProbabilisticActor, TanhNormal

# Loss
from torchrl.objectives import ClipPPOLoss, ValueEstimators

# Utils
torch.manual_seed(0)
from matplotlib import pyplot as plt
from tqdm import tqdm

In [150]:
from torchrl.envs import PettingZooWrapper

group_map = {
    "scout": ["player_0"],
    "guards": ["player_1", "player_2", "player_3"]
}

raw_env = gridworld.env()
env = PettingZooWrapper(raw_env, use_mask=True, group_map=group_map)


In [151]:
is_fork = multiprocessing.get_start_method() == "fork"
device = (
    torch.device(0)
    if torch.cuda.is_available() and not is_fork
    else torch.device("cpu")
)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#Parameters for Env
n_parallel_envs = 2  # Number of parallel environments

# Sampling
frames_per_batch = 2_000  # Number of team frames collected per training iteration
total_frames = 200_000

# Training
num_epochs = 5  # Number of optimization steps per training iteration
minibatch_size = 400  # Size of the mini-batches in each optimization step
lr = 3e-4  # Learning rate
max_grad_norm = 1.0  # Maximum norm for the gradients

# PPO
clip_epsilon = 0.2  # clip value for PPO loss
gamma = 0.99  # discount factor
lmbda = 0.9  # lambda for generalised advantage estimation
entropy_eps = 1e-4  # coefficient of the entropy term in the PPO loss

# disable log-prob aggregation
set_composite_lp_aggregate(False).set()

In [152]:
env.rollout(5)

TensorDict(
    fields={
        done: Tensor(shape=torch.Size([5, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        guards: TensorDict(
            fields={
                action: Tensor(shape=torch.Size([5, 3]), device=cpu, dtype=torch.int64, is_shared=False),
                done: Tensor(shape=torch.Size([5, 3, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                mask: Tensor(shape=torch.Size([5, 3]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([5, 3, 576]), device=cpu, dtype=torch.int64, is_shared=False),
                terminated: Tensor(shape=torch.Size([5, 3, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([5, 3, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([5, 3]),
            device=None,
            is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(sh

### Transform

In [156]:
agent_names = ["scout", "guards"]

# Create RewardSum transforms for each agent
reward_transforms = [
    RewardSum(
        in_keys=[(agent, "reward")],
        out_keys=[(agent, "episode_reward")]
    ) for agent in agent_names
]

# Compose all transforms
env_transforms = Compose(
    *reward_transforms,
    DoubleToFloat(),
    StepCounter()
)

# Set up the environment creation function
make_env = EnvCreator(lambda: TransformedEnv(
    PettingZooWrapper(raw_env, use_mask=True, group_map=group_map),  # call raw_env() if it's a function
    Compose(
    *reward_transforms,
    DoubleToFloat(),
    StepCounter()
)
))

# Create parallel environments
env = ParallelEnv(n_parallel_envs, make_env, serial_for_single=True)


In [157]:
agent_names = ["player_0", "player_1", "player_2", "player_3"]

# Create RewardSum transforms for each agent
reward_transforms = [
    RewardSum(
        in_keys=[(agent, "reward")],
        out_keys=[(agent, "episode_reward")]
    ) for agent in agent_names
]

# Compose all transforms
env_transforms = Compose(
    *reward_transforms,
    DoubleToFloat(),
    StepCounter()
)

# Set up the environment creation function
make_env = EnvCreator(lambda: TransformedEnv(
    PettingZooWrapper(raw_env, use_mask=True),  # call raw_env() if it's a function
    Compose(
    *reward_transforms,
    DoubleToFloat(),
    StepCounter()
)
))

# Create parallel environments
check_env = ParallelEnv(n_parallel_envs, make_env, serial_for_single=True)

In [158]:
# print("action_keys:", env.action_keys)
# print("reward_keys:", env.reward_keys)
# print("done_keys:", env.done_keys)

# print("Action Spec:", env.action_spec)
# print("Observation Spec:", env.observation_spec)
# print("Reward Spec:", env.reward_spec)
# print("Done Spec:", env.done_spec)

check_env_specs(check_env)

/home/ubuntu/Desktop/real_learning/venv/lib/python3.12/site-packages/torchrl/data/replay_buffers/samplers.py:37: UserWarning: Failed to import torchrl C++ binaries. Some modules (eg, prioritized replay buffers) may not work with your installation. This is likely due to a discrepancy between your package version and the PyTorch version. Make sure both are compatible. Usually, torchrl majors follow the pytorch majors within a few days around the release. For instance, TorchRL 0.5 requires PyTorch 2.4.0, and TorchRL 0.6 requires PyTorch 2.5.0.
  warnings.warn(EXTENSION_WARNING)
/home/ubuntu/Desktop/real_learning/venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
Process _ProcessNoWarn-2

EOFError: 

/home/ubuntu/Desktop/real_learning/venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
Process _ProcessNoWarn-24:
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/ubuntu/Desktop/real_learning/venv/lib/python3.12/site-packages/torchrl/_utils.py", line 734, in run
    return mp.Process.run(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/Desktop/real_learning/venv/lib/python3.12/site-packages/torchrl/envs/batched_envs.py", line 216

In [ ]:
n_rollout_steps = 5
rollout = env.rollout(n_rollout_steps)
print(f"rollout of {n_rollout_steps} steps:", rollout)
print("Shape of the rollout TensorDict:", rollout.batch_size)

rollout of 5 steps: TensorDict(
    fields={
        done: Tensor(shape=torch.Size([2, 5, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([2, 5, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                player_0: TensorDict(
                    fields={
                        done: Tensor(shape=torch.Size([2, 5, 1, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                        episode_reward: Tensor(shape=torch.Size([2, 5, 1, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                        mask: Tensor(shape=torch.Size([2, 5, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                        observation: Tensor(shape=torch.Size([2, 5, 1, 576]), device=cpu, dtype=torch.int64, is_shared=False),
                        reward: Tensor(shape=torch.Size([2, 5, 1, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                        ter

### Assign policies

In [104]:
n_player_0 = env.observation_spec["player_0", "observation"].shape[1]
n_player_1 = env.observation_spec["player_1", "observation"].shape[1]
n_player_2 = env.observation_spec["player_2", "observation"].shape[1]
n_player_3 = env.observation_spec["player_3", "observation"].shape[1]
n_entities = env.observation_spec["player_0", "observation"].shape[1]
n_features = env.observation_spec["player_0", "observation"].shape[2]
# n_features = env.observation_spec["player_0", "observation"].shape[3]
print(env.observation_spec["player_0", "observation"])
# print(n_entities)

BoundedDiscrete(
    shape=torch.Size([2, 1, 576]),
    space=ContinuousBox(
        low=Tensor(shape=torch.Size([2, 1, 576]), device=cpu, dtype=torch.int64, contiguous=True),
        high=Tensor(shape=torch.Size([2, 1, 576]), device=cpu, dtype=torch.int64, contiguous=True)),
    device=cpu,
    dtype=torch.int64,
    domain=discrete)


In [105]:
print(env.observation_spec["player_3", "observation"])

BoundedDiscrete(
    shape=torch.Size([2, 1, 576]),
    space=ContinuousBox(
        low=Tensor(shape=torch.Size([2, 1, 576]), device=cpu, dtype=torch.int64, contiguous=True),
        high=Tensor(shape=torch.Size([2, 1, 576]), device=cpu, dtype=torch.int64, contiguous=True)),
    device=cpu,
    dtype=torch.int64,
    domain=discrete)


In [106]:
# Create a flattening module that handles the batched_env+time dimensions
class FlattenObs(nn.Module):
    def forward(self, obs):
        # Convert to float first
        obs = obs.float()

        # Handle different possible shapes
        if len(obs.shape) == 5:  # [batch_env, time, n_agents, n_entities, n_features]
            batch_env, time, n_agents, n_entities, n_features = obs.shape

            # Reshape to merge batch_env and time dimensions
            # This gives [batch_env*time, n_agents, n_entities, n_features]
            obs = obs.reshape(-1, n_agents, n_entities, n_features)

            # Take only the first entity for each agent (agent itself)
            return obs[:, :, 0, :]  # [batch_env*time, n_agents, n_features]

        elif len(obs.shape) == 4:  # [batch, n_agents, n_entities, n_features]
            batch, n_agents, n_entities, n_features = obs.shape
            return obs[:, :, 0, :]  # [batch, n_agents, n_features]

        elif len(obs.shape) == 3:  # [batch, n_entities, n_features]
              batch, n_entities, n_features = obs.shape
              return obs.reshape(batch, n_entities * n_features)

        # Fallback for unexpected shapes
        return obs

In [111]:
policy_modules = {}
critics = {}

obs_dim = 576
n_actions = 5

# Scout policy (player_0)
policy_mlp_scout = MultiAgentMLP(
    n_agent_inputs=obs_dim,
    n_agent_outputs=n_actions,
    n_agents=1,  # 1 scout
    centralised=False,
    share_params=True,
    device=device,
    depth=2,
    num_cells=256,
    activation_class=nn.Tanh,
)

# Guards policy (players 1-3)
policy_mlp_guard = MultiAgentMLP(
    n_agent_inputs=obs_dim,
    n_agent_outputs=n_actions,
    n_agents=3,  # 3 guards
    centralised=False,
    share_params=True,  # Share params among guards
    device=device,
    depth=2,
    num_cells=256,
    activation_class=nn.Tanh,
)

# Create sequential modules with flattening if needed
policy_seq_scout = nn.Sequential(policy_mlp_scout, nn.Softmax(dim=-1))
policy_seq_guard = nn.Sequential(policy_mlp_guard, nn.Softmax(dim=-1))

# Wrap in TensorDictModule
policy_modules["player_0"] = TensorDictModule(
    module=policy_seq_scout,
    in_keys=[("player_0", "observation")],
    out_keys=[("player_0", "probs")]
)

for i in range(1, 4):
    policy_modules[f"player_{i}"] = TensorDictModule(
        module=policy_seq_guard,
        in_keys=[(f"player_{i}", "observation")],
        out_keys=[(f"player_{i}", "probs")]
    )

In [115]:
print(env.observation_spec["player_2", "observation"])

BoundedDiscrete(
    shape=torch.Size([2, 1, 576]),
    space=ContinuousBox(
        low=Tensor(shape=torch.Size([2, 1, 576]), device=cpu, dtype=torch.int64, contiguous=True),
        high=Tensor(shape=torch.Size([2, 1, 576]), device=cpu, dtype=torch.int64, contiguous=True)),
    device=cpu,
    dtype=torch.int64,
    domain=discrete)


### Critic

In [116]:
# Scout critic
MAPPO = True

critic_module_scout = TensorDictModule(
    module=MultiAgentMLP(
        n_agent_inputs=obs_dim,
        n_agent_outputs=1,
        n_agents=1,
        centralised=MAPPO,  # True for MAPPO, False for IPPO
        share_params=True,
        device=device,
        activation_class=nn.Tanh,
        depth=2,
        num_cells=256,
    ),
    in_keys=[("player_0", "observation")],
    out_keys=[("player_0", "state_value")]
)

# Guards critic
critic_module_guard = TensorDictModule(
    module=MultiAgentMLP(
        n_agent_inputs=obs_dim,
        n_agent_outputs=1,
        n_agents=3,
        centralised=MAPPO,  # True for MAPPO, False for IPPO
        share_params=True,  # Share among guards
        device=device,
        activation_class=nn.Tanh,
        depth=2,
        num_cells=256,
    ),
    in_keys=[("player_1", "observation")],  # Will be applied to all guards
    out_keys=[("player_1", "state_value")]
)

critics["player_0"] = critic_module_scout

# Use same critic for all guards
for i in range(1, 4):
    critics[f"player_{i}"] = critic_module_guard

In [117]:
policies = {}
for i in range(4):
    player = f"player_{i}"
    policies[player] = ProbabilisticActor(
        module=policy_modules[player],
        spec=env.action_spec[player, "action"],
        in_keys=[(player, "probs")],
        out_keys=[(player, "action")],
        distribution_class=Categorical,
        return_log_prob=True,
    )

In [118]:
# Convert the policy dictionary to a sequential policy for the collector

# If you're using the actors dictionary as your policy:
agents_policy = TensorDictSequential(*policies.values())

# Or if you have a separate policies dictionary:
# agents_policy = TensorDictSequential(*policies.values())

# Create the collector with the sequential policy
collector = SyncDataCollector(
    env,
    agents_policy,
    device=device,
    storing_device=device,
    frames_per_batch=frames_per_batch,
    total_frames=total_frames,
)

RuntimeError: Failed while executing module '0'. Scroll up for more info.

In [ ]:
# Define agent groups manually
groups = ["archer", "knight"]

# Create separate replay buffers for each group
replay_buffers = {}
for group in groups: 
    replay_buffers[group] = ReplayBuffer(
        storage=LazyTensorStorage(
            frames_per_batch, device=device
        ),
        sampler=SamplerWithoutReplacement(),
        batch_size=minibatch_size,
    )

In [ ]:
def process_batch(batch: TensorDictBase) -> TensorDictBase:
    """Expand done and terminated keys for each group to match reward shape."""
    for i in range(4):
        player = f"player_{i}"
        keys = list(batch.keys(True, True))
        group_shape = batch.get_item_shape(player)
        nested_done_key = ("next", player, "done")
        nested_terminated_key = ("next", player, "terminated")
        if nested_done_key not in keys:
            batch.set(
                nested_done_key,
                batch.get(("next", "done")).unsqueeze(-1).expand((*group_shape, 1)),
            )
        if nested_terminated_key not in keys:
            batch.set(
                nested_terminated_key,
                batch.get(("next", "terminated"))
                .unsqueeze(-1)
                .expand((*group_shape, 1)),
            )
    return batch

# Training loop
groups = [f"player_{i}" for i in range(4)]  # All 4 players
episode_reward_mean_map = {group: [] for group in groups}

for iteration, batch in enumerate(collector):
    batch = process_batch(batch)
    
    # Process each player
    for group in groups:
        # Extract data for this player only
        group_batch = batch.exclude(
            *[
                key
                for player in groups
                if player != group
                for key in [player, ("next", player)]
            ]
        )
        
        # Reshape and add to replay buffer
        group_batch = group_batch.reshape(-1)
        replay_buffers[group].extend(group_batch)
        
        # PPO training epochs
        for _ in range(num_epochs):
            for subdata in replay_buffers[group]:
                loss_vals = losses[group](subdata)
                loss_value = (
                    loss_vals["loss_objective"] +
                    loss_vals["loss_critic"] +
                    loss_vals["loss_entropy"]
                )
                
                optimizers[group].zero_grad()
                loss_value.backward()
                torch.nn.utils.clip_grad_norm_(
                    losses[group].parameters(), max_grad_norm
                )
                optimizers[group].step()
    
    # Update policy weights
    collector.update_policy_weights_()
    
    # Logging
    for group in groups:
        done_mask = batch.get(("next", group, "done"))
        if done_mask.any():
            episode_reward_mean = (
                batch.get(("next", group, "episode_reward"))[done_mask]
                .mean()
                .item()
            )
        else:
            episode_reward_mean = (
                episode_reward_mean_map[group][-1] if episode_reward_mean_map[group] else 0.0
            )
        episode_reward_mean_map[group].append(episode_reward_mean)

In [109]:
env = TransformedEnv(
    raw_env,
    Compose(
        # normalize observations
        ObservationNorm(in_keys=["observation"]),
        DoubleToFloat(),
        StepCounter(),
    ),
)


AttributeError: 'raw_env' object has no attribute 'device'

In [ ]:
from torchrl.objectives import ClipPPOLoss
from torch.optim import Adam

obs_dim = 4
action_dim = 5

# Initialize networks
policy_net = PolicyNetwork(obs_dim, action_dim)
value_net = ValueNetwork(obs_dim)

# Define optimizer
optimizer = Adam(list(policy_net.parameters()) + list(value_net.parameters()), lr=3e-4)

# Define PPO loss
loss_fn = ClipPPOLoss(policy_net, value_net)


AttributeError: 'PolicyNetwork' object has no attribute 'log_prob_keys'

In [ ]:
for episode in range(num_episodes):
    obs = wrapped_env.reset()
    done = False
    while not done:
        # Select action using policy network
        action = policy_net(obs)
        # Step the environment
        next_obs, reward, done, info = wrapped_env.step(action)
        # Compute loss and update networks
        loss = loss_fn(obs, action, reward, next_obs, done)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        obs = next_obs


# Ray PPO

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.registry import register_env
from ray import tune

raw_env = gridworld.env()
register_env("my_env", raw_env)

config = (
    PPOConfig()
    # Set the config object's env.
    .environment(env="my_env")
    # Update the config object's training parameters.
    .training(
        lr=0.001, clip_param=0.2
    )
)

tune.Tuner(
    "PPO",
    run_config=tune.RunConfig(stop={"training_iteration": 1}),
    param_space=config,
).fit()

TypeError: ('Second argument must be callable.', <pettingzoo.utils.wrappers.order_enforcing.OrderEnforcingWrapper object at 0x710749eb2fc0>)

# TensorFlow Model

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

2025-06-03 10:04:40.196828: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-03 10:04:40.205249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748916280.214868   74622 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748916280.217930   74622 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748916280.227155   74622 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [8]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='relu'))
    return model

# Agent

In [5]:
import torch.nn as nn
import torch
from torch.distributions import Categorical
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, input_channels=7, input_dim=35, hidden_dim=128, depth=2):
        super().__init__()
        self.feature_extractor = nn.ModuleList()
        
        # Determine per-channel output dim such that total is close to hidden_dim
        per_channel_dim = hidden_dim // input_channels
        
        for _ in range(input_channels):
            layers = []
            dim = input_dim
            for _ in range(depth - 1):
                layers.append(nn.Linear(dim, per_channel_dim))
                layers.append(nn.ReLU())
                dim = per_channel_dim
            # Ensure the final layer matches per_channel_dim (in case depth=1)
            layers.append(nn.Linear(dim, per_channel_dim))
            layers.append(nn.ReLU())
            self.feature_extractor.append(nn.Sequential(*layers))

        # The concatenated feature will have dimension input_channels * per_channel_dim ≈ hidden_dim
        self.fc = nn.Linear(input_channels * per_channel_dim, hidden_dim)
        
        layers = []
        for _ in range(depth):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        # x shape: [batch_size, input_channels, input_dim]
        features = [self.feature_extractor[i](x[:, i, :]) for i in range(len(self.feature_extractor))]
        x = torch.cat(features, dim=-1)
        x = self.fc(x)
        x = F.relu(x)
        x = self.net(x)
        return x

    
class ActorHead(nn.Module):
    def __init__(self, hidden_dim, action_space, depth):
        super().__init__()
        layers = []
        in_dim = hidden_dim
        for _ in range(depth - 1):
            layers.append(nn.Linear(in_dim, hidden_dim))
            layers.append(nn.ReLU())
            in_dim = hidden_dim
        # Final layer outputs logits over actions
        layers.append(nn.Linear(in_dim, action_space))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

class Agent(nn.Module):
    def __init__(self, input_channels, input_dim, hidden_dim, depth, action_space, temperature):
        super().__init__()
        self.encoder = Encoder(
                        input_channels=input_channels,
                        input_dim=input_dim,
                        hidden_dim=hidden_dim,
                        depth=depth
                        )
        self.actor = ActorHead(
                        hidden_dim=hidden_dim,
                        action_space=action_space,
                        depth=depth)
        self.temperature = temperature
    
    def forward(self, x, deterministic=False):
        encoded = self.encoder(x)     
        logits = self.actor(encoded)             
        if deterministic:
            action = torch.argmax(logits, dim=-1)  # Choose most probable action          
        else:      
            scaled_logits = logits / self.temperature
            dist = Categorical(logits=scaled_logits)
            action = dist.sample()                   
        return action

# Wrapper

In [6]:
import functools
from pettingzoo.utils.env import ActionType, AECEnv, AgentID, ObsType
from pettingzoo.utils.wrappers.base import BaseWrapper

class CustomWrapper(BaseWrapper[AgentID, ObsType, ActionType]):
    def __init__(
        self,
        env: AECEnv[AgentID, ObsType, ActionType],
    ):
        super().__init__(env)

    def reset(self, seed=None, options=None):
        super().reset(seed, options)

    def step(self, action: ActionType):
        super().step(action)

    def observe(self, agent: AgentID) -> ObsType | None:
        obs = super().observe(agent)
        return obs

    @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        space = super().observation_space(agent)
        return space